In [20]:
!pip install awswrangler
!pip install skforecast
!pip install pmdarima

In [21]:
import awswrangler as wr
import pandas as pd
import boto3
import pickle
from io import BytesIO
from io import StringIO
import joblib
import os
from datetime import datetime, timedelta
import warnings

# skforecast
from skforecast.Sarimax import Sarimax
from skforecast.ForecasterSarimax import ForecasterSarimax
from skforecast.model_selection_sarimax import backtesting_sarimax
from skforecast.model_selection_sarimax import grid_search_sarimax
from sklearn.metrics import mean_absolute_error


### Load ABT and .pkl

In [22]:
# Set up S3 client
client = boto3.client('s3')
bucket_name = 'viamericas-datalake-dev-us-east-1-283731589572-analytics'
path = 'ABTv3/Last_releases_7d_8lags/' # Folder under analysis

#ABTversion = 'v3_update'
ABTversion = 'v6'
path_ABT = f'ABT{ABTversion}/' 

In [23]:
# Specify the CSV file key
csv_key = None

# List objects in the S3 path
response = client.list_objects(Bucket=bucket_name, Prefix=path_ABT)

# Find the CSV file in the S3 path
for obj in response.get('Contents', []):
    if obj['Key'].endswith('.csv'):
        csv_key = obj['Key']
        break
        
# Check if CSV file is found
if csv_key is not None:
    # Read CSV content from S3
    csv_response = client.get_object(Bucket=bucket_name, Key=csv_key)
    csv_content = csv_response['Body'].read().decode('utf-8')

    # Transform CSV content to DataFrame
    data = pd.read_csv(StringIO(csv_content))
    data['date'] = pd.to_datetime(data['date']).dt.date
    print("CSV file loaded")
else:
    print("No CSV file found in the specified S3 path.")

CSV file loaded


In [24]:
data['date'] = pd.to_datetime(data['date'])

In [25]:
# Mark with 1 in 'day_of_the_dead' when 'date' is Nov. 2
data.loc[data['date'].dt.month.eq(11) & data['date'].dt.day.eq(2), 'day_of_the_dead'] = 1

In [26]:
data['amount'].sum()

27791166635.5917

In [27]:
data.date.max()

Timestamp('2024-02-11 00:00:00')

In [28]:
data.payer_country.nunique()

133

## LOAD .pkl

In [29]:
elements = client.list_objects(Bucket=bucket_name, Prefix=path)

#Listing pkl files
pkl_files = [obj['Key'] for obj in elements.get('Contents', []) if obj['Key'].endswith('.pkl')
                and ('MODEL_7d' in obj['Key']) and not obj['Key'].endswith('_2d.pkl')]  # File structure 

### NO OLVIDAR QUE EN EL CASO A 2D HAY QUE LEVANTAR 15 MODELOS DE V3_UPDATE

In [30]:
len(pkl_files)

131

In [ ]:
pkl_files

In [31]:
today = datetime.now().date()

### Loop to process all

In [32]:
# Initialize an empty DataFrame to store the results
df_temp = pd.DataFrame(columns=['date', 'pred', 'payer_country', 'model'])

In [33]:
# Payers to present errors in the inference process.
payer_countries_pinched = []

In [ ]:
#data=data.loc[data.payer_country=='ELEKTRA (MEXICO)_MEXICO']

In [35]:
i = 1

# Iterate over pkl files
for file_key in pkl_files:
    # Extract payer_country from file_key
    payer_country = file_key.split('/')[2]
    print(payer_country)
    
    # Download pkl file from S3 and load it into memory
    response = client.get_object(Bucket=bucket_name, Key=file_key)
    buffer = BytesIO(response['Body'].read())
    forecaster = joblib.load(buffer)
    
    #### PAYER SETTING ####
    # Filter data for the specific payer_country
    datos = data[data['payer_country'] == payer_country].copy()
#    datos = data.loc[data.payer_country == payer_country]
    datos['date'] = pd.to_datetime(datos['date'])
    datos.set_index('date', inplace=True)
    datos = datos.asfreq('D')

    # Predictions settings
    last_window_date = forecaster.last_window.index[-1] + pd.Timedelta(days=1) #19/12/23 first day in last window
    test_date = forecaster.last_window.index[-1] + pd.Timedelta(days=48) # The first test day would be the day to predict 29/12/23-==11-(esto CRECE SIEMPRE UN DIA MAS)

    # Extract data for last window and test period
    data_last_window = datos.loc[last_window_date:test_date - pd.Timedelta(days=1)].copy() #From 19/12/23 to test minus one day 28/12/23
    data_last_window[forecaster.exog_col_names] = data_last_window[forecaster.exog_col_names].fillna(0)##extracting exogenous and filling missing
    data_test = datos.loc[test_date:test_date + pd.Timedelta(days=8)].copy()#29/12/2023 we have data in test (first day to predict) and their extension (2 days) (7D IN 8D MODEL)
    data_test[forecaster.exog_col_names] = data_test[forecaster.exog_col_names].fillna(0)##we fill in missings

    try:
        
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
        # Make predictions
        predictions = forecaster.predict(
                          steps            = 8, # Days to predict
                          exog             = data_test[forecaster.exog_col_names],
                          last_window      = data_last_window['amount'],
                          last_window_exog = data_last_window[forecaster.exog_col_names]
                      )
        # Store predictions in a temporary DataFrame
        df_temp = pd.DataFrame(predictions, columns=['pred']).reset_index()

    except:
        # If an exception occurs, set predictions to zero
        print("\033[1;31m" + f"Error processing {payer_country}" + "\033[0m")
        predictions = [0, 0]
        df_temp = pd.DataFrame({'index': [test_date, test_date + pd.Timedelta(days=1)], 'pred': predictions})
        payer_countries_pinched.append(payer_country)


    # Add additional columns
    df_temp['payer_country'] = payer_country
    df_temp['model'] = file_key.split('/')[-1]

    # Concatenate df_temp with the main DataFrame
    if i:
        temp_df = df_temp.copy()
        i = 0
    else:
        temp_df = pd.concat([temp_df, df_temp], ignore_index=True)

24XORO_MEXICO
ABANK (TN)_EL SALVADOR
AFEX_CHILE
AFRO INTERNACIONAL_GUINEA
AFRO INTERNACIONAL_SIERRA LEONE
AIRPAK_MEXICO
ARGENPER_ARGENTINA
ARGENPER_BOLIVIA
ARGENPER_CHILE
ARGENPER_PERU
BAM - BANCO AGROMERCANTIL (UT)_GUATEMALA
BANCO AGRICOLA_EL SALVADOR
BANCO ATLANTIDA_HONDURAS
BANCO BHD LEON_DOMINICAN REPUBLIC
BANCO BISA_BOLIVIA
BANCO DAVIVIENDA SALVADORENO (BTS)_EL SALVADOR
BANCO DAYCOVAL_BRAZIL
BANCO DE CREDITO DEL PERU - BCP (UT)_PERU
BANCO DE GUAYAQUIL_ECUADOR
BANCO DE OCCIDENTE_HONDURAS
BANCO DE ORO (BDO)_PHILIPPINES
BANCO DEL AUSTRO_ECUADOR
BANCO GANADERO (BOLIVIA)_BOLIVIA
BANCO INDUSTRIAL ELS (RED CHAPINA)_EL SALVADOR
BANCO INDUSTRIAL_GUATEMALA
BANCO PICHINCHA (TN)_ECUADOR
BANCO POPULAR HONDURAS (UT)_HONDURAS
BANCO RENDIMENTO_BRAZIL
BANCOLOMBIA_COLOMBIA
BANCOPPEL (APPRIZA)_MEXICO
BANHCAFE_HONDURAS
BANK OF PHILIPPINE ISLANDS (BPI)_PHILIPPINES
BANORTE (UT)_MEXICO
BANPAIS (RED CHAPINA)_HONDURAS
BANPRO_NICARAGUA
BANRURAL (HONDURAS)_HONDURAS
BANRURAL (RYT)_GUATEMALA
BANTRAB (UT)_GUAT

In [ ]:
#print(pkl_files)
stop

In [36]:
data_last_window.index.min()
#test_date

Timestamp('2023-12-19 00:00:00')

In [ ]:
payer_countries_pinched

In [37]:
temp_df[temp_df['pred'] < 0]

,index,pred,payer_country,model
208,2024-01-25,-184.123648,BANCO POPULAR HONDURAS (UT)_HONDURAS,MODEL_7dBANCO POPULAR HONDURAS (UT)_HONDURAS.pkl
212,2024-01-29,-40.048305,BANCO POPULAR HONDURAS (UT)_HONDURAS,MODEL_7dBANCO POPULAR HONDURAS (UT)_HONDURAS.pkl
213,2024-01-30,-120.512884,BANCO POPULAR HONDURAS (UT)_HONDURAS,MODEL_7dBANCO POPULAR HONDURAS (UT)_HONDURAS.pkl
214,2024-01-31,-153.487272,BANCO POPULAR HONDURAS (UT)_HONDURAS,MODEL_7dBANCO POPULAR HONDURAS (UT)_HONDURAS.pkl
215,2024-02-01,-176.684666,BANCO POPULAR HONDURAS (UT)_HONDURAS,MODEL_7dBANCO POPULAR HONDURAS (UT)_HONDURAS.pkl
320,2024-02-04,-53.555528,BNB_SENEGAL,MODEL_7dBNB_SENEGAL.pkl
321,2024-02-05,-108.375092,BNB_SENEGAL,MODEL_7dBNB_SENEGAL.pkl
322,2024-02-06,-164.629078,BNB_SENEGAL,MODEL_7dBNB_SENEGAL.pkl
323,2024-02-07,-210.278328,BNB_SENEGAL,MODEL_7dBNB_SENEGAL.pkl
324,2024-02-08,-244.369926,BNB_SENEGAL,MODEL_7dBNB_SENEGAL.pkl


In [38]:
# Replace values in 'pred' column with 0 where 'pred' is less than 0
temp_df.loc[temp_df['pred'] < 0, 'pred'] = 0

In [39]:
# Convert 'index' column to date format
temp_df['index'] = pd.to_datetime(temp_df['index']).dt.date

In [40]:
temp_df.head(4)

,index,pred,payer_country,model
0,2024-02-04,51071.713833,24XORO_MEXICO,MODEL_7d24XORO_MEXICO.pkl
1,2024-02-05,36777.216823,24XORO_MEXICO,MODEL_7d24XORO_MEXICO.pkl
2,2024-02-06,29308.021520,24XORO_MEXICO,MODEL_7d24XORO_MEXICO.pkl
3,2024-02-07,34602.889308,24XORO_MEXICO,MODEL_7d24XORO_MEXICO.pkl


In [41]:
temp_df.rename(columns={'index':'pred_date'}, inplace=True)

In [42]:
temp_df

,pred_date,pred,payer_country,model
0,2024-02-04,51071.713833,24XORO_MEXICO,MODEL_7d24XORO_MEXICO.pkl
1,2024-02-05,36777.216823,24XORO_MEXICO,MODEL_7d24XORO_MEXICO.pkl
2,2024-02-06,29308.021520,24XORO_MEXICO,MODEL_7d24XORO_MEXICO.pkl
3,2024-02-07,34602.889308,24XORO_MEXICO,MODEL_7d24XORO_MEXICO.pkl
4,2024-02-08,30823.064887,24XORO_MEXICO,MODEL_7d24XORO_MEXICO.pkl
...,...,...,...,...
1037,2024-02-07,0.000000,ZEEPAY_GHANA,MODEL_7dZEEPAY_GHANA.pkl
1038,2024-02-08,0.000000,ZEEPAY_GHANA,MODEL_7dZEEPAY_GHANA.pkl
1039,2024-02-09,947.553815,ZEEPAY_GHANA,MODEL_7dZEEPAY_GHANA.pkl
1040,2024-02-10,0.000000,ZEEPAY_GHANA,MODEL_7dZEEPAY_GHANA.pkl


In [43]:
control=temp_df.loc[temp_df.payer_country=="ELEKTRA (MEXICO)_MEXICO"]

In [44]:
control

,pred_date,pred,payer_country,model
456,2024-02-04,8.704538e+06,ELEKTRA (MEXICO)_MEXICO,MODEL_7dELEKTRA (MEXICO)_MEXICO.pkl
457,2024-02-05,7.760511e+06,ELEKTRA (MEXICO)_MEXICO,MODEL_7dELEKTRA (MEXICO)_MEXICO.pkl
458,2024-02-06,6.872001e+06,ELEKTRA (MEXICO)_MEXICO,MODEL_7dELEKTRA (MEXICO)_MEXICO.pkl
459,2024-02-07,6.255463e+06,ELEKTRA (MEXICO)_MEXICO,MODEL_7dELEKTRA (MEXICO)_MEXICO.pkl
460,2024-02-08,6.702523e+06,ELEKTRA (MEXICO)_MEXICO,MODEL_7dELEKTRA (MEXICO)_MEXICO.pkl
461,2024-02-09,9.432478e+06,ELEKTRA (MEXICO)_MEXICO,MODEL_7dELEKTRA (MEXICO)_MEXICO.pkl
462,2024-02-10,1.045332e+07,ELEKTRA (MEXICO)_MEXICO,MODEL_7dELEKTRA (MEXICO)_MEXICO.pkl
463,2024-02-11,8.795229e+06,ELEKTRA (MEXICO)_MEXICO,MODEL_7dELEKTRA (MEXICO)_MEXICO.pkl


In [45]:
control=temp_df.loc[temp_df.payer_country=="BANCOPPEL (APPRIZA)_MEXICO"]

In [46]:
control

,pred_date,pred,payer_country,model
232,2024-02-04,5.332340e+06,BANCOPPEL (APPRIZA)_MEXICO,MODEL_7dBANCOPPEL (APPRIZA)_MEXICO.pkl
233,2024-02-05,4.367202e+06,BANCOPPEL (APPRIZA)_MEXICO,MODEL_7dBANCOPPEL (APPRIZA)_MEXICO.pkl
234,2024-02-06,3.755596e+06,BANCOPPEL (APPRIZA)_MEXICO,MODEL_7dBANCOPPEL (APPRIZA)_MEXICO.pkl
235,2024-02-07,3.393576e+06,BANCOPPEL (APPRIZA)_MEXICO,MODEL_7dBANCOPPEL (APPRIZA)_MEXICO.pkl
236,2024-02-08,3.669053e+06,BANCOPPEL (APPRIZA)_MEXICO,MODEL_7dBANCOPPEL (APPRIZA)_MEXICO.pkl
237,2024-02-09,5.579573e+06,BANCOPPEL (APPRIZA)_MEXICO,MODEL_7dBANCOPPEL (APPRIZA)_MEXICO.pkl
238,2024-02-10,6.127768e+06,BANCOPPEL (APPRIZA)_MEXICO,MODEL_7dBANCOPPEL (APPRIZA)_MEXICO.pkl
239,2024-02-11,5.104473e+06,BANCOPPEL (APPRIZA)_MEXICO,MODEL_7dBANCOPPEL (APPRIZA)_MEXICO.pkl


In [47]:
control=temp_df.loc[temp_df.payer_country=="BANRURAL (RYT)_GUATEMALA"]

In [48]:
control

,pred_date,pred,payer_country,model
288,2024-02-04,5.461759e+06,BANRURAL (RYT)_GUATEMALA,MODEL_7dBANRURAL (RYT)_GUATEMALA.pkl
289,2024-02-05,3.713901e+06,BANRURAL (RYT)_GUATEMALA,MODEL_7dBANRURAL (RYT)_GUATEMALA.pkl
290,2024-02-06,2.936361e+06,BANRURAL (RYT)_GUATEMALA,MODEL_7dBANRURAL (RYT)_GUATEMALA.pkl
291,2024-02-07,2.625472e+06,BANRURAL (RYT)_GUATEMALA,MODEL_7dBANRURAL (RYT)_GUATEMALA.pkl
292,2024-02-08,2.815353e+06,BANRURAL (RYT)_GUATEMALA,MODEL_7dBANRURAL (RYT)_GUATEMALA.pkl
293,2024-02-09,4.645762e+06,BANRURAL (RYT)_GUATEMALA,MODEL_7dBANRURAL (RYT)_GUATEMALA.pkl
294,2024-02-10,5.352956e+06,BANRURAL (RYT)_GUATEMALA,MODEL_7dBANRURAL (RYT)_GUATEMALA.pkl
295,2024-02-11,5.292901e+06,BANRURAL (RYT)_GUATEMALA,MODEL_7dBANRURAL (RYT)_GUATEMALA.pkl


In [ ]:
stop

### ADD ID_COUNTRY AND ID_PAYER FROM DAILY_CHECK_GP TABLE

In [ ]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'
origin_name = 'AwsDataCatalog'
database_name= 'analytics'
table_name = 'daily_check_gp'

In [ ]:
df = wr.athena.read_sql_table(
    table=table_name,
    database=database_name,
)

In [ ]:
df['payer_country'] = df['payer'] + '_' + df['country']

In [ ]:
df_id = df[['payer_country', 'id_main_branch', 'id_country']].drop_duplicates().dropna(subset='id_main_branch')

In [ ]:
len(df_id)

In [ ]:
df_final = pd.merge(temp_df, df_id, on='payer_country', how='left')

In [ ]:
# Insertar la columna 'processing_date' al principio del DataFrame
df_final.insert(0, 'processing_date', today)

In [ ]:
df_final['processing_date'] = pd.to_datetime(df_final['processing_date'] )

In [ ]:
df_final['id_country'].isnull().sum()

In [ ]:
df_final['id_main_branch'].isnull().sum()

In [ ]:
# Splitting 'payer' & 'country'
df_final[['payer', 'country']] = df_final['payer_country'].str.split('_', expand=True)

In [ ]:
df_final.head()

In [ ]:
df_final.info()

In [ ]:
df_final.pred.sum()

### SAVE THE INFERENCES OF JANUARY 28TH AND 29TH
##### THIS IS THE INPUT FOR ADDING MAPES TO HISTORIC MAPES EXCEL

In [ ]:
bucket_name = 'viamericas-datalake-dev-us-east-1-283731589572-analytics'
path_inference = '/ABTv3_update/' # Folder under analysis

In [ ]:
#wr.s3.to_csv(
    df=df_final,
    path='s3://viamericas-datalake-dev-us-east-1-283731589572-analytics/ABTv3_update/Inferences_2d/predictions_2d.csv',
    dataset=False,
    index=False
)

In [ ]:
control=df_final.loc[df_final.payer_country=="GIROSMEX_MEXICO"]

In [ ]:
control